In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from sympy.physics.vector import dynamicsymbols

In [2]:
class Transformation:
    
    @staticmethod
    def rotate_x(angle: int | float | sp.Symbol) -> sp.Matrix:
        rot = sp.eye(4, 4)
        rot[1:3, 1:3] = sp.Matrix([
            [sp.cos(angle), -sp.sin(angle)],
            [sp.sin(angle), sp.cos(angle)]
        ])
        return rot
    
    @staticmethod
    def rotate_z(angle: int | float | sp.Symbol) -> sp.Matrix:
        rot = sp.eye(4, 4)
        rot[0:2, 0:2] = sp.Matrix([
            [sp.cos(angle), -sp.sin(angle)],
            [sp.sin(angle), sp.cos(angle)]
        ])
        return rot
    
    @staticmethod
    def rotate_y(angle: int | float | sp.Symbol) -> sp.Matrix:
        rot = sp.eye(4, 4)
        rot[0, 0:3] = sp.Matrix([sp.cos(angle), 0, sp.sin(angle)]).T
        rot[2, 0:3] = sp.Matrix([-sp.sin(angle), 0, sp.cos(angle)]).T
        return rot
    
    @staticmethod
    def translate(pos: sp.Matrix | np.ndarray | list) -> sp.Matrix:
        rot = sp.eye(4, 4)
        rot[0:3, 3] = pos
        return rot

## Forward Kinematic solution for Puma 560


![](./forward.jpg)


For wrist part of manipulator we will use Euler angles `ZXZ`

Then we will have the FK like that:

$$
T(q_{1-6}) = R_z(q_1) T_z(L_1) R_y(-\frac{\pi}{2}) R_z(q_2) T_z(L_2) T_x(-L_3) R_z(q_3) T_z(-L_4) T_x(-L_5) R_y(\frac{\pi}{2}) R_z(q_4) R_x(q_5) R_z(q_6) 
$$

In [103]:
# Lenghts of the arm
l = sp.symbols("L_{1:6}")

# Dynamic parameters of rotation
q = dynamicsymbols("q_{1:7}")

T = sp.simplify(
    Transformation.rotate_z(q[0]) @ Transformation.rotate_y(- sp.pi / 2) \
        @ Transformation.rotate_z(q[1]) @ Transformation.translate([0, 0, l[1]]) @ Transformation.translate([-l[2], 0, 0]) \
            @ Transformation.rotate_z(q[2]) @ Transformation.translate([0, 0, -l[3]]) @ Transformation.translate([-l[4], 0, 0]) @ Transformation.rotate_y(sp.pi / 2) \
                @ Transformation.rotate_z(q[3]) @ Transformation.rotate_x(q[4]) @ Transformation.rotate_z(q[5])
)

### Rotation matrix of FK

In [104]:
l_val = [0, 0.128, 0.432, 0.089, 0.432]

R = T[0:3, 0:3]
P = T[0:3, 3]

R

Matrix([
[-((sin(q_{1}(t))*cos(q_{2}(t) + q_{3}(t))*cos(q_{4}(t)) + sin(q_{4}(t))*cos(q_{1}(t)))*cos(q_{5}(t)) + sin(q_{2}(t) + q_{3}(t))*sin(q_{1}(t))*sin(q_{5}(t)))*sin(q_{6}(t)) - (sin(q_{1}(t))*sin(q_{4}(t))*cos(q_{2}(t) + q_{3}(t)) - cos(q_{1}(t))*cos(q_{4}(t)))*cos(q_{6}(t)), -((sin(q_{1}(t))*cos(q_{2}(t) + q_{3}(t))*cos(q_{4}(t)) + sin(q_{4}(t))*cos(q_{1}(t)))*cos(q_{5}(t)) + sin(q_{2}(t) + q_{3}(t))*sin(q_{1}(t))*sin(q_{5}(t)))*cos(q_{6}(t)) + (sin(q_{1}(t))*sin(q_{4}(t))*cos(q_{2}(t) + q_{3}(t)) - cos(q_{1}(t))*cos(q_{4}(t)))*sin(q_{6}(t)), (sin(q_{1}(t))*cos(q_{2}(t) + q_{3}(t))*cos(q_{4}(t)) + sin(q_{4}(t))*cos(q_{1}(t)))*sin(q_{5}(t)) - sin(q_{2}(t) + q_{3}(t))*sin(q_{1}(t))*cos(q_{5}(t))],
[-((sin(q_{1}(t))*sin(q_{4}(t)) - cos(q_{2}(t) + q_{3}(t))*cos(q_{1}(t))*cos(q_{4}(t)))*cos(q_{5}(t)) - sin(q_{2}(t) + q_{3}(t))*sin(q_{5}(t))*cos(q_{1}(t)))*sin(q_{6}(t)) + (sin(q_{1}(t))*cos(q_{4}(t)) + sin(q_{4}(t))*cos(q_{2}(t) + q_{3}(t))*cos(q_{1}(t)))*cos(q_{6}(t)), -((sin(q_{1}(t

### Transition vector of FK

In [105]:
P

Matrix([
[-L_{2}*cos(q_{1}(t)) + L_{3}*sin(q_{1}(t))*sin(q_{2}(t)) + L_{4}*cos(q_{1}(t)) + L_{5}*sin(q_{2}(t) + q_{3}(t))*sin(q_{1}(t))],
[-L_{2}*sin(q_{1}(t)) - L_{3}*sin(q_{2}(t))*cos(q_{1}(t)) + L_{4}*sin(q_{1}(t)) - L_{5}*sin(q_{2}(t) + q_{3}(t))*cos(q_{1}(t))],
[                                                                        -L_{3}*cos(q_{2}(t)) - L_{5}*cos(q_{2}(t) + q_{3}(t))]])

In [106]:
FK = sp.lambdify(q, T.subs(zip(l, l_val)), "numpy")

In [107]:
fk_sol = FK(np.pi/3, np.pi/6, np.pi/6, np.pi/4, np.pi/12, np.pi/12)

fk_sol

array([[-0.1694222 , -0.81530496, -0.55369119,  0.49156149],
       [ 0.68236694, -0.50241001,  0.53099863, -0.32883648],
       [-0.71110581, -0.28785761,  0.64145656, -0.59012297],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

## Solution of Inverse Kinematic (IK)

![](./inverse.jpg)

We have target position $p_{target} = \begin{bmatrix} x \\ y \\ z \end{bmatrix}$ and rotation matrix $R_{target} = \begin{bmatrix} R_{11} & R_{12} & R_{13} \\ R_{21} & R_{22} & R_{23} \\ R_{31} & R_{32} & R_{33} \end{bmatrix}$

In [115]:
sp.simplify(Transformation.rotate_z(q[3]) @ Transformation.rotate_x(q[4]) @ Transformation.rotate_z(q[5])) [0:3, 0:3]

Matrix([
[-sin(q_{4}(t))*sin(q_{6}(t))*cos(q_{5}(t)) + cos(q_{4}(t))*cos(q_{6}(t)), -sin(q_{4}(t))*cos(q_{5}(t))*cos(q_{6}(t)) - sin(q_{6}(t))*cos(q_{4}(t)),  sin(q_{4}(t))*sin(q_{5}(t))],
[ sin(q_{4}(t))*cos(q_{6}(t)) + sin(q_{6}(t))*cos(q_{4}(t))*cos(q_{5}(t)), -sin(q_{4}(t))*sin(q_{6}(t)) + cos(q_{4}(t))*cos(q_{5}(t))*cos(q_{6}(t)), -sin(q_{5}(t))*cos(q_{4}(t))],
[                                             sin(q_{5}(t))*sin(q_{6}(t)),                                              sin(q_{5}(t))*cos(q_{6}(t)),                cos(q_{5}(t))]])

In [145]:
from math import atan, acos
def IK(mat):
    r = mat[0:3, 0:3]
    x, y, z = mat[0:3, 3]
    
    el, wr, d = l_val[2], l_val[4], l_val[1] 
    
    L1 = np.sqrt(x ** 2 + y ** 2)
    L2 = np.sqrt(L1 ** 2 - d ** 2)
    L3 = np.sqrt(L2 ** 2 + z ** 2)
    
    phi2 = np.atan(x / y)
    phi3 = np.atan(L2 / d)
    
    theta2 = np.atan(z / L2)
    theta3 = np.arccos((L3 ** 2 + el ** 2 - wr ** 2) / (2 * L3 * el))
    
    psi = 2 * np.pi - np.arccos((-L3 ** 2 + el ** 2 + wr ** 2) / (2 * wr * el))  # q3
    theta = 2 * np.pi + (theta2 - theta3)  # q2
    phi = np.pi - (phi2 + phi3)  # q1
    
    r_sol = np.array(Transformation.rotate_z(phi) @ Transformation.rotate_z(theta) @ Transformation.rotate_z(psi))[0:3, 0:3].T @ r
    
    q4 = atan(r_sol[0, 2] / - r_sol[1, 2])
    q5 = acos(r_sol[2, 2])
    q6 = atan(r_sol[2, 0] / r_sol[2, 1])
    return np.array([phi, theta, psi, q4, q5, q6])

In [147]:
qs = IK(fk_sol)

In [148]:
fk_sol

array([[-0.1694222 , -0.81530496, -0.55369119,  0.49156149],
       [ 0.68236694, -0.50241001,  0.53099863, -0.32883648],
       [-0.71110581, -0.28785761,  0.64145656, -0.59012297],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [149]:
FK(*qs)

array([[-0.80978961,  0.11889212,  0.57454805, -0.02830866],
       [-0.34228624,  0.69961183, -0.62720285, -0.18012984],
       [-0.47653009, -0.70456224, -0.52584326,  0.18204838],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [150]:
np.rad2deg(qs)

array([158.71848161, 297.2306881 , 214.28918039, -84.03985192,
        50.09948238,  67.96176061])

In [151]:
np.rad2deg([np.pi/3, np.pi/6, np.pi/6, np.pi/4, np.pi/12, np.pi/12])

array([60., 30., 30., 45., 15., 15.])